# Football Forecasting

Football is ubiquitous.The main objective of this project is to forecast football outcomes.... more flavor. 

We recreate the proposals of [Google](https://github.com/GoogleCloudPlatform/ipython-soccer-predictions) and [Goldman Sachs](http://www.goldmansachs.com/our-thinking/macroeconomic-insights/euro-cup-2016/) to tackle this task. We implemented both algortihms to predict the results of Comebol qualifiers and estimate the probability that Peru qualifies to the World Cup. Then, we propose a different approach in order to improve the accuaracy of the model. 


## Data
The database contains information at match level from seasons 2011/12, 2012/13 and 2013/14 for three leagues and the 2014 World Cup. The leagues included are:
* MLS (USA)
* Premier League (England)
* La Liga (Spain)
 
Attributes used for prediction consider the average of the last six games of each team (home and away).

In [3]:
import pandas as pd
import numpy as np
import math
import time
import pylab as pl
import matplotlib.pyplot as plt
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)

import code.wc_pred.world_cup as world_cup
import code.wc_pred.match_stats as match_stats
import code.wc_pred.power as power
import code.wc_pred.odds as odds
import code.algorithms.config as config

# Import main dataset
parser2 = lambda date: pd.datetime.strptime(date, '%Y-%m-%d %H:%M:%f')
raw_data = pd.read_csv('./data/raw_data_ready.csv', index_col=0, header=0, 
                       parse_dates=['timestamp'], date_parser=parser2, 
                       encoding='utf-8')

# Import Comebol dataset
test_peru = pd.read_csv('./data/comebol_complete.csv', index_col=0, header=0)

# Partition the main dataset and the club data. We're only going to 
# train our model using club data.
club_data = raw_data.loc[raw_data['competitionid'] != 4]

    
# Generate a table with goals and points using club data.
points = club_data.replace({'points': {0: 'lose', 1: 'tie', 3: 'win'}})['points']
goals_points = pd.crosstab(club_data['goals'], points)

print('Goals and points:\n{0}'.format(goals_points))
print('\nPoints frequency:\n{0}'.format(points.value_counts()))
print('\nGoals frequency:\n{0}'.format(club_data['goals'].value_counts()))


Goals and points:
points  lose  tie  win
goals                 
0        727  267    0
1        477  394  320
2        131  205  500
3         21   40  314
4          2    6  148
5          0    2   65
6          0    0   12
7          0    0    6
8          0    0    1

Points frequency:
win     1366
lose    1358
tie      914
Name: points, dtype: int64

Goals frequency:
1    1191
0     994
2     836
3     375
4     156
5      67
6      12
7       6
8       1
Name: goals, dtype: int64
